In [1]:
import torch
import torch.nn as nn

In [2]:
# dir(nn.Module)

In [3]:
#importing libraries 
import numpy as np
import pandas as pd

from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import cv2

from tqdm import tqdm_notebook as tqdm
from functools import partial
import scipy as sp

import random
import time
import sys
import os
import yaml

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from sklearn.metrics import confusion_matrix
import torch
import torchvision

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms, models, datasets
from torchvision.models.feature_extraction import create_feature_extractor
from torch.utils.data import Dataset
from torch.autograd import Variable

import warnings
warnings.filterwarnings('ignore')

In [4]:
train_on_gpu = torch.cuda.is_available()
if not train_on_gpu:
    print('CUDA is not available. Training on CPU...')
    device = torch.device('cpu')
else:
    print('CUDA is available. Training on GPU...')
    device = torch.device('cuda:0')

CUDA is available. Training on GPU...


In [15]:
config_path = 'config/preprocessing_config.yaml'
conf = yaml.safe_load(open(config_path,'r'))
train_image_path = conf['train_dataset']['image_path']
test_image_path = conf['test_dataset']['image_path']
train_labels_path = conf['train_dataset']['label_path']
test_labels_path = conf['test_dataset']['label_path']
batch_size = conf['image_preprocessing']['batch_size']
shuffle = conf['image_preprocessing']['shuffle']
do_random_crop = conf['image_preprocessing']['do_random_crop']
columns = conf['image_preprocessing']['columns']
image_size = conf['image_preprocessing']['image_size']
itype = conf['image_preprocessing']['itype']
device = conf['metadata']['device']

In [16]:
from utils.dataloader import load_data
from utils.functions import *

In [17]:
# import data
train = pd.read_csv(train_labels_path)
test = pd.read_csv(test_labels_path)

# check shape
print(train.shape, test.shape)
print('-' * 15)
print(train[columns[1]].value_counts())
print('-' * 15)
print(test[columns[1]].value_counts())

(413, 12) (103, 3)
---------------
Retinopathy grade
2    136
0    134
3     74
4     49
1     20
Name: count, dtype: int64
---------------
Retinopathy grade
0    34
2    32
3    19
4    13
1     5
Name: count, dtype: int64


In [18]:
loader = load_data(train_labels_path, test_labels_path, train_image_path, test_image_path, columns, itype = '.jpg', batch_size = 16, shuffle=True, do_random_crop = False, device = 'cpu')
train_loader, test_loader, valid_loader = loader.create_loader()

In [19]:
# count = 0
# for batch in train_loader:
#     print(batch['image'].shape)
#     print(batch['label'].shape)
#     count = count+1
# print(count)
# count=0
# for batch in test_loader:
#     print(batch['image'].shape)
#     print(batch['label'].shape)
#     count=count+1
# print(count)
# count=0
# for batch in valid_loader:
#     print(batch['image'].shape)
#     print(batch['label'].shape)
#     count=count+1
# print(count)
# count=0

In [20]:
print(loader.valid_df.iloc[:5, 0:2])

   Image name  Retinopathy grade
51  IDRiD_052                  0
52  IDRiD_053                  4
53  IDRiD_054                  0
54  IDRiD_055                  0
55  IDRiD_056                  0


In [21]:
class RetinopathyClassification(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            
            nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
        
            nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            # nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            # nn.ReLU(inplace=True),
            # nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            # nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            
            nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            # nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            # nn.ReLU(inplace=True),
            # nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            # nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False),
            
            nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.ReLU(inplace=True),
            # nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            # nn.ReLU(inplace=True),
            # nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            # nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
        
        self.avgpool = nn.Sequential(
                      
            nn.AdaptiveAvgPool2d(output_size=(7, 7))
        )
    
        self.classifier = nn.Sequential(
            nn.Flatten(),  
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=4096, bias=True),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5, inplace=False),
            nn.Linear(in_features=4096, out_features=1000, bias=True),
            nn.ReLU(inplace=True),
            nn.Linear(in_features=1000, out_features=5, bias=True),
            nn.Softmax()
        )
        
        self.overall = nn.Sequential(
            self.features,
            self.avgpool,
            self.classifier
        )
        
    def forward(self, xb):
        
        self.out = self.overall(xb)
        return self.out

In [22]:
epochs = 10
lr = 1e-4
model = RetinopathyClassification().to(device)

: 

: 

In [13]:
def fit(epochs, lr, model, train_loader, opt_func = torch.optim.Adam):
    
    history = []
    optimizer = opt_func(model.parameters(),lr)
    for epoch in range(epochs):
        
        model.train()
        result = {'train_loss': [], 'train_acc': []}
        train_losses = []
        train_accuracy = []
        for batch in train_loader:
            images, labels = batch['image'], batch['label']
            optimizer.zero_grad()
            out = model(images)                  # Generate predictions
            loss = ordinal_loss(out, labels) # Calculate loss
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            acc = ordinal_accuracy(out, labels)
            train_accuracy.append(acc)
            print("Epoch [{}], Batch [{}], train_loss: {:.4f}, train_acc: {:.4f}, val_acc: {:.4f}".format(
            epoch, batch, loss.detach(), acc.detach()), flush=True)
            result['train_loss'].append(loss.detach())
            result['train_acc'].append(acc.detach())
            
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['train_acc'] = torch.stack(train_accuracy).mean().item()
        print("Epoch [{}], train_loss: {:.4f}, train_acc: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['train_acc']))
        history.append(result)
    
    return history

In [14]:
fit(epochs, lr, model, train_loader)

RuntimeError: output with shape [] doesn't match the broadcast shape [16]